## ***2023 Brain Age Predition - iFLYTEK A.I. Developer Competition***

In [8]:
!pip uninstall matplotlib

In [1]:
import pandas as pd
import numpy as np
import sklearn
import torch

import missingno as msno

In f:\Anaconda\ANA\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In f:\Anaconda\ANA\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In f:\Anaconda\ANA\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In f:\Anaconda\ANA\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In f:\Anaconda\ANA\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.jpeg_quality rcparam was 

#### ***1 Load Dataset***

***This Dataset contains 12 csv files, which come from MRI scanner.***

***This part aims to load the dataset and use pre-process methods to construct data features***

In [74]:
import os 
import numpy as np

train_datafolder_path = 'data\\train'
test_datafolder_path = 'data\\test'

def read_datafile(datafolder_path, is_test=False):
    feature_dict = {}
    for file_name in os.listdir(datafolder_path):
        data_path = os.path.join(datafolder_path, file_name)
        feature_df = pd.read_csv(data_path)
        feature_name = file_name[:file_name.find('-')]
        print(feature_name)

        if 'subject_info' in data_path:
            if not is_test:
                # get label colunm from y_dataframe
                label_df = feature_df.iloc[:, 3]
                feature_dict[feature_name] = label_df
            else:
                feature_dict[feature_name] = None
        else:
            # drop the label numbers of x_dataframe
            feature_df = feature_df.iloc[:, 1:]
            feature_dict[feature_name] = feature_df

    return feature_dict

train_feature_dict, test_feature_dict = read_datafile(train_datafolder_path), read_datafile(test_datafolder_path, is_test=True)

print(train_feature_dict.keys())

aseg
lh.GausCurv
lh.GrayVol
lh.MeanCurv
lh.SurfArea
lh.ThickAvg
rh.GausCurv
rh.GrayVol
rh.MeanCurv
rh.SurfArea
rh.ThickAvg
subject_info
wmparc
aseg
lh.GausCurv
lh.GrayVol
lh.MeanCurv
lh.SurfArea
rh.GausCurv
rh.GrayVol
rh.MeanCurv
rh.SurfArea
rh.ThickAvg
subject_info
wmparc
dict_keys(['aseg', 'lh.GausCurv', 'lh.GrayVol', 'lh.MeanCurv', 'lh.SurfArea', 'lh.ThickAvg', 'rh.GausCurv', 'rh.GrayVol', 'rh.MeanCurv', 'rh.SurfArea', 'rh.ThickAvg', 'subject_info', 'wmparc'])


In [ ]:
import matplotlib.pyplot as plt

# use missingno to check the dataframe
for index, name in enumerate(train_feature_dict.keys()):
    if name != 'subject_info':
        msno.matrix(train_feature_dict[name], figsize=(15, 5))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# use seaborm to visualize the distribution
for index, name in enumerate(train_feature_dict.keys()):
    if name != 'subject_info':
        sns.distplot(train_feature_dict[name], bins=100, color="green") 
        plt.show()

In [77]:

def data_preprocess(data_dict):
    # Min-Max
    for index, feature_df in enumerate(data_dict.keys()):
        if name != 'subject_info':
            data_dict[feature_df] = (data_dict[feature_df] - data_dict[feature_df].min()) / (data_dict[feature_df].max() - data_dict[feature_df].min())
    return data_dict

def feature_aug(data_dict):

    return None

def feature_combination(data_dict):

    original_x_thick = (data_dict['lh.ThickAvg'] + data_dict['rh.ThickAvg']) * 0.5
    original_x_gauscurv = (data_dict['lh.GausCurv'] + data_dict['rh.GausCurv']) * 0.5
    original_x_meancurv = (data_dict['lh.MeanCurv'] + data_dict['rh.MeanCurv']) * 0.5
    original_x_surfarea = (data_dict['lh.SurfArea'] + data_dict['rh.SurfArea']) * 0.5
    original_x_grayvol = (data_dict['lh.GrayVol'] + data_dict['rh.GrayVol']) * 0.5
    original_x_wmparc = data_dict['wmparc']
    original_x_aseg = data_dict['aseg']

    original_x = np.hstack(
        [original_x_thick, original_x_gauscurv, original_x_meancurv, original_x_surfarea, original_x_grayvol, original_x_wmparc, original_x_aseg, 
         data_dict['lh.ThickAvg'], data_dict['rh.ThickAvg'], data_dict['lh.GausCurv'], data_dict['rh.GausCurv'], data_dict['lh.MeanCurv'],
         data_dict['rh.MeanCurv'], data_dict['lh.SurfArea'], data_dict['rh.SurfArea'], data_dict['lh.GrayVol'], data_dict['rh.GrayVol']])
    
    return original_x

original_x = feature_combination(data_preprocess(train_feature_dict))
print(original_x)

[[0.80438696 0.83953257 0.53569915 ... 0.67346087 0.54215385 0.77105575]
 [0.94836407 0.78379198 0.59376558 ... 0.59944348 0.53661538 0.66429419]
 [0.73413451 0.77679294 0.49010547 ... 0.70921739 0.55815385 0.86081455]
 ...
 [0.69663256 0.69873467 0.49867682 ... 0.57892174 0.48246154 0.6915777 ]
 [0.75346653 0.62082909 0.39679729 ... 0.66518261 0.648      0.8413075 ]
 [0.74340632 0.56885192 0.3168239  ... 0.54761739 0.58953846 0.63094767]]


### ***2 Models***

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from RVM import *

In [ ]:


# 将数据集分为训练集和测试集
X_train, X_valid, y_train, y_valid = train_test_split(train_original_x, train_original_y, test_size=0.3, random_state=0)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_valid = torch.tensor(X_train, dtype=torch.float32)
y_valid = torch.tensor(y_train, dtype=torch.float32)
test_original_x = torch.tensor(test_original_x, dtype=torch.float32)

# RVR
print('\n RVR Model')

linear_model = RVR(kernel="linear")
linear_model.fit(X_train, y_train)
linear_model_valid_prediction = linear_model.predict(X_valid)
print('\nVALID MAE:{}'.format(mean_absolute_error(y_valid, [int(i) for i in linear_model_valid_prediction])))
print('VALID Variance {}'.format(np.var([int(i) for i in linear_model_valid_prediction])))
linear_model_test_prediction = linear_model.predict(test_original_x)
print([int(i) for i in linear_model_test_prediction])
print('TEST Variance {}'.format(np.var([int(i) for i in linear_model_test_prediction])))

rbf_model = RVR(kernel="rbf")
rbf_model.fit(X_train, y_train)
rbf_model_valid_prediction = rbf_model.predict(X_valid)
print('\nVALID MAE:{}'.format(mean_absolute_error(y_valid, [int(i) for i in rbf_model_valid_prediction])))
print('VALID Variance {}'.format(np.var([int(i) for i in rbf_model_valid_prediction])))
rbf_model_test_prediction = rbf_model.predict(test_original_x)
print([int(i) for i in rbf_model_test_prediction])
print('TEST Variance {}'.format(np.var([int(i) for i in rbf_model_test_prediction])))

poly_model = RVR(kernel="poly")
poly_model.fit(X_train, y_train)
poly_model_valid_prediction = poly_model.predict(X_valid)
print('\nVALID MAE:{}'.format(mean_absolute_error(y_valid, [int(i) for i in poly_model_valid_prediction])))
print('VALID Variance {}'.format(np.var([int(i) for i in poly_model_valid_prediction])))
poly_model_test_prediction = rbf_model.predict(test_original_x)
print([int(i) for i in poly_model_test_prediction])
print('TEST Variance {}'.format(np.var([int(i) for i in poly_model_test_prediction])))

# 随机森林
print('\n RandomForest Model')

rf_model = RandomForestRegressor(n_estimators=12, random_state=0)
rf_model.fit(X_train, y_train)
rf_model_valid_prediction = rf_model.predict(X_valid)
print('\nVALID MAE:{}'.format(mean_absolute_error(y_valid, [int(i) for i in rf_model_valid_prediction])))
print('VALID Variance {}'.format(np.var([int(i) for i in rf_model_valid_prediction])))
rf_model_test_prediction = rf_model.predict(test_original_x)
print([int(i) for i in rf_model_test_prediction])
print('TEST Variance {}'.format(np.var([int(i) for i in rf_model_test_prediction])))

# Bagging
print('\n Bagging Model')

bag_model = BaggingRegressor()
bag_model.fit(X_train, y_train)
bag_model_valid_prediction = bag_model.predict(X_valid)
print('\nVALID MAE:{}'.format(mean_absolute_error(y_valid, [int(i) for i in bag_model_valid_prediction])))
print('VALID Variance {}'.format(np.var([int(i) for i in bag_model_valid_prediction])))
bag_model_test_prediction = bag_model.predict(test_original_x)
print([int(i) for i in bag_model_test_prediction])
print('TEST Variance {}'.format(np.var([int(i) for i in bag_model_test_prediction])))

# XGBoost
print('\n XGBoost Model')

xgb_model = XGBRegressor(learning_rate=0.2)
xgb_model.fit(X_train, y_train)
xgb_model_valid_prediction = xgb_model.predict(X_valid)
print('\nVALID MAE:{}'.format(mean_absolute_error(y_valid, [int(i) for i in xgb_model_valid_prediction])))
print('LABEL Variance {}'.format(np.var([int(i) for i in y_valid])))
print('VALID Variance {}'.format(np.var([int(i) for i in xgb_model_valid_prediction])))
xgb_model_test_prediction = xgb_model.predict(test_original_x)
print([int(i) for i in xgb_model_test_prediction])
print('TEST Variance {}'.format(np.var([int(i) for i in xgb_model_test_prediction])))

# LightGBM
print('\n LightGBM Model')

lgbm_model = LGBMRegressor()
lgbm_model.fit(X_train, y_train)
lgbm_model_valid_prediction = lgbm_model.predict(X_valid)
print('\nVALID MAE:{}'.format(mean_absolute_error(y_valid, [int(i) for i in lgbm_model_valid_prediction])))
print('LABEL Variance {}'.format(np.var([int(i) for i in y_valid])))
print('VALID Variance {}'.format(np.var([int(i) for i in lgbm_model_valid_prediction])))
lgbm_model_test_prediction = lgbm_model.predict(test_original_x)
print([int(i) for i in lgbm_model_test_prediction])
print('TEST Variance {}'.format(np.var([int(i) for i in lgbm_model_test_prediction])))

# 集成模型
print('\n Ensemble Model')

new_linear_pred, new_rbf_pred, new_poly_pred = [int(i) for i in linear_model_valid_prediction], [int(i) for i in rbf_model_valid_prediction], [int(i) for i in poly_model_valid_prediction]
new_rf_pred = [int(i) for i in rf_model_valid_prediction]
new_bag_pred = [int(i) for i in bag_model_valid_prediction]
new_xgb_pred = [int(i) for i in xgb_model_valid_prediction]
new_lgbm_pred = [int(i) for i in lgbm_model_valid_prediction]

# ensemble_prediction = [(i + j) / 2 for i, j in zip(new_rf_pred, new_rbf_pred)]
ensemble_prediction = [(i + j + k) / 3 for i, j, k in zip(new_xgb_pred, new_bag_pred, new_linear_pred)]
print('\nENSEMBLE VALID MAE:{}'.format(mean_absolute_error(y_valid, ensemble_prediction)))
print('ENSEMBLE VALID Variance {}'.format(np.var([int(i) for i in ensemble_prediction])))

test_new_linear_pred, test_new_rbf_pred, test_new_poly_pred = [int(i) for i in linear_model_test_prediction], [
    int(i) for i in rbf_model_test_prediction], [int(i) for i in poly_model_test_prediction]
test_new_bag_pred = [int(i) for i in bag_model_test_prediction]
test_new_rf_pred = [int(i) for i in rf_model_test_prediction]
test_new_xgb_pref = [int(i) for i in xgb_model_test_prediction]

# test_ensemble_prediction = [(i + j) / 2 for i, j in zip(test_new_rf_pred, test_new_rbf_pred)]
test_ensemble_prediction = [(i + j + k) / 3 for i, j, k in
                            zip(test_new_xgb_pref, test_new_bag_pred, test_new_linear_pred)]
print([int(i) for i in test_ensemble_prediction])
print('TEST Variance {}'.format(np.var([int(i) for i in test_ensemble_prediction])))

# 写结果
# final_list = [int(i) for i in [int(i) for i in lgbm_model_test_prediction]]
# submission_version_name = 'Ensemble-DataAug3-XGBoost_Bagging_RVR'
# write_submission(final_list, submission_version_name)
